In [31]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [32]:
GRID_DIM = 5
GRID_SIZE = GRID_DIM * GRID_DIM
A, A_ , B, B_ = np.array([0, 1]), np.array([4, 1]), np.array([0, 3]), np.array([2, 3])
LEFT, RIGHT, UP , DOWN = np.array([0, -1]), np.array([0, 1]), np.array([-1, 0]), \
                        np.array([1, 0]) 
A_REWARD = +10.0
B_REWARD = +5.0
GAMMA = 0.9 
Pi_as = 0.25
ITERATIONS = 10000
ACTIONS = [LEFT, RIGHT , UP , DOWN]

def get_value(i, j):
    reward = 0.0
    for action in ACTIONS:
        cur_location = np.array([i, j]) 
        next_location = cur_location +  action
        reward += get_reward(cur_location, next_location)
    return Pi_as * reward
def get_reward(location, n_location):
    if location[0] == A[0] and location[1] == A[1]:
        return A_REWARD
    if location[0] == B[0] and location[1] == B[1]:
        return B_REWARD
    
    if n_location[0] >= GRID_DIM or n_location[1] < 0:
        return -1
    else:
        return 0
def get_state_reward(cur_state, action):
    if cur_state[0] == A[0] and cur_state[1] == A[1]:
        return A_, A_REWARD
    if cur_state[0] == B[0] and cur_state[1] == B[1]:
        return B_, B_REWARD
    
    n_loc = cur_state + action
    if n_loc[0] >= GRID_DIM or n_loc[0] < 0 or n_loc[1] >= GRID_DIM or n_loc[1] < 0:
        return cur_state, -1
    else:
        return n_loc, 0

In [33]:
b = []
for i in range(GRID_DIM):
    for j in range(GRID_DIM):
        for action in ACTIONS:
            b.append(get_reward((i,j), action))
b = -np.array(b)

In [34]:
a_mat = np.zeros((GRID_DIM*GRID_DIM*len(ACTIONS), GRID_DIM*GRID_DIM)) 

j = 0
for i in range(0,GRID_DIM*GRID_DIM*len(ACTIONS),len(ACTIONS)):
    for k in range(len(ACTIONS)):        
        a_mat[i+k][j] = -1
    j+=1
    if j == 25:
        break
        
for i in range(GRID_DIM):
    for j in range(GRID_DIM):
        for action_index, action in enumerate(ACTIONS):
            cur_loc = np.array([i,j])
            st, rew = get_state_reward(cur_loc, action)
            cur_row = i * GRID_DIM * len(ACTIONS)
            cur_row += j * len(ACTIONS)+ action_index
            cur_col = st[0]*GRID_DIM + st[1]
            a_mat[cur_row, cur_col] += GAMMA
    

In [35]:
import scipy.optimize as opt

In [36]:
print(a_mat.shape, b.shape)
print(b)

(100, 25) (100,)
[  1.  -0.  -0.  -0. -10. -10. -10. -10.   1.  -0.  -0.  -0.  -5.  -5.
  -5.  -5.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.
   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.
  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.
   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.
  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.
   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.  -0.  -0.   1.  -0.
  -0.  -0.]


In [40]:
obj = np.ones((GRID_DIM*GRID_DIM))
x = opt.linprog(obj, a_mat, b)

# print(np.round(x['x'], decimals=1).reshape(5,5))

v_pi_star = x['x']
print('Vpi')
print(v_pi_star.reshape(5,5))
# print(x)

Vpi
[[21.97748529 24.4194281  20.97748529 18.4501845  15.60516605]
 [19.77973676 21.97748529 18.87973676 16.60516605 14.04464945]
 [17.80176308 19.77973676 16.99176308 14.94464945 12.6401845 ]
 [16.02158677 17.80176308 15.29258677 13.4501845  11.37616605]
 [14.4194281  16.02158677 13.7633281  12.10516605 10.23854945]]


In [38]:
pi_ = np.zeros((GRID_DIM, GRID_DIM))
v_pi_star  = np.around(v_pi_star.reshape(5,5), decimals=1)
vp = []
for i in range(GRID_DIM):
    for j in range(GRID_DIM):
        values = []
        act = []
        for action in ACTIONS:
            st, rew = get_state_reward(np.array([i,j]), action)
            val = v_pi_star[st[0], st[1]]
            values.append(np.around(val, decimals=1))
        max_value = max(values)
        
        for i, value in enumerate(values):
            if value == max_value:
                 act.append(i)
        vp.append(act)
        pi_[i, j] = np.argmax(values)

        

In [39]:
for i in range(GRID_DIM):
    print()
    for j in range(GRID_DIM):
        print(vp[i*GRID_DIM + j], '|' , end =" ")


[1] | [2] | [0] | [0] | [0] | 
[1, 2] | [2] | [0] | [0] | [0] | 
[1, 2] | [2] | [0] | [0] | [0] | 
[1, 2] | [2] | [0] | [0] | [0] | 
[1, 2] | [2] | [0] | [0] | [0] | 